In [10]:
##import modules
import os
import psycopg2 as psy
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd


In [11]:
##set working directory
ficher = os.environ.get("FICHER")
os.chdir(ficher+"/Projects/one_mbps_state_breakdown")

## db credentials
HOST_PROD = os.environ.get("HOST_DAR")
USER_PROD = os.environ.get("USER_DAR")
PASSWORD_PROD = os.environ.get("PASSWORD_DAR")
DB_PROD = os.environ.get("DB_DAR")
print(ficher)
print(HOST_PROD)

/Users/chriskemnitzer/Documents/SAT/ficher/
ec2-34-232-141-208.compute-1.amazonaws.com


In [12]:
##Query db for current frn info
myConnection = psy.connect(dbname=DB_PROD, user=USER_PROD, password=PASSWORD_PROD, host=HOST_PROD, port='5432')

cur = myConnection.cursor()

def query_script(file):
    sql = open(file)
    query = sql.read()
    cur.execute(query)
    names = [x[0] for x in cur.description]
    rows = cur.fetchall()
    df = pd.DataFrame(rows, columns=names)
    sql.close()
    return df
states_1mbps_df = query_script("state_level_1mbps.sql")

In [13]:
current_states_df = states_1mbps_df[(states_1mbps_df['funding_year'] == 2019)]

##converting columns to numeric
numeric_cols = ['pct_districts_meeting_1mbps','pct_students_meeting_1mbps','median_ia_bandwidth_per_student_kbps','median_ia_monthly_cost_per_mbps','increase_pct_dist_meeting_1mbps','max_year_pct_increase','pct_rural','pct_small']

current_states_df[numeric_cols] = current_states_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [14]:
##adding column for state meeting goals categories
def goal_cat(row):
    if row['pct_districts_meeting_1mbps'] >= .5:
        return 'Majority Meeting'
    elif row['pct_districts_meeting_1mbps'] < .5 and row['pct_districts_meeting_1mbps'] >= .2:
        return 'Normal'
    elif row['pct_districts_meeting_1mbps'] < .2:
        return 'Lagging'
    else: 
        return None
current_states_df['meeting_category'] = current_states_df[['pct_districts_meeting_1mbps']].apply(goal_cat, axis=1)

## adding column for growth category 
def growth_cat(row):
    if row['max_year_pct_increase'] >= .2:
        return 'High Growth'
    elif row['max_year_pct_increase'] >= .1:
        return 'Normal'
    elif row['max_year_pct_increase'] < .1:
        return 'Lagging'
    else:
        return None
current_states_df['growth_category'] = current_states_df[['max_year_pct_increase']].apply(growth_cat, axis=1)


##define model state if >50% of districts are meeting or they have had a high growth year
def model_bool(row):
    if (row['meeting_category'] == 'Majority Meeting') | (row['growth_category'] == 'High Growth'):
        return True
    else:
        return False
    
current_states_df['model_state'] = current_states_df[['meeting_category','growth_category']].apply(model_bool, axis=1)

##define states that have not made progress towards 1 mbps goal
def lag_bool(row):
    if (row['meeting_category'] == 'Lagging' and row['growth_category'] == 'Lagging'):
        return True
    else:
        return False
    
current_states_df['lagging_state'] = current_states_df[['meeting_category','growth_category']].apply(lag_bool, axis=1)

##rounding long decimals 
decimals = pd.Series( ([3]*len(numeric_cols)) , index=numeric_cols)
current_states_df = current_states_df.round(decimals)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [15]:
model_states_df = current_states_df[(current_states_df['model_state'] == True)]
model_states_df.head()

lagging_states_df = current_states_df[(current_states_df['lagging_state'] == True)]
lagging_states_df.head()

example_states_df = current_states_df[(current_states_df['model_state'] == True) |(current_states_df['lagging_state'] == True)]
example_states_df

,state_code,funding_year,districts_meeting_2018_goal_oversub,districts_not_meeting_2018_goal_oversub,pct_districts_meeting_1mbps,increase_pct_dist_meeting_1mbps,max_year_pct_increase,students_meeting_2018_goal_oversub,students_not_meeting_2018_goal_oversub,pct_students_meeting_1mbps,...,districts_population,students_population,pct_rural,pct_small,med_pct_of_meeting_goal,procurement_pattern,meeting_category,growth_category,model_state,lagging_state
4,AK,2019,4.15686274509803921552,48.84313725490196078448,0.078,0.002,0.053,383.88219728232661381714,121883.11780271767338618286,0.003,...,53,122267,0.906,0.623,0.25641025641025641030,District-procured,Lagging,Lagging,False,True
9,AL,2019,11.63793103448275862055,123.36206896551724137945,0.086,0.064,0.064,25360.37595230899104924558,710458.62404769100895075442,0.034,...,135,735819,0.696,0.393,0.47877453280132767100,State Procured,Lagging,Lagging,False,True
14,AR,2019,234.00000000000000000000,0E-20,1.000,0.022,0.919,473462.00000000000000000000,None,1.000,...,234,473462,0.872,0.821,1.9197323428840939,State Procured,Majority Meeting,High Growth,True,False
44,DE,2019,7.00000000000000000006,9.99999999999999999994,0.412,0.099,0.333,40300.99999999999999959172,72677.00000000000000040828,0.357,...,17,112978,0.471,0.294,0.76745970836531082120,State Procured,Normal,High Growth,True,False
49,FL,2019,2.00000000000000000014,64.99999999999999999986,0.030,0.000,0.033,45081.99999999999999260136,2485160.00000000000000739864,0.018,...,67,2530242,0.507,0.149,0.27583553210202286720,District-procured,Lagging,Lagging,False,True
54,GA,2019,9.00000000000000000000,171.00000000000000000000,0.050,0.006,0.015,8578.60011611878811744949,1705432.39988388121188255051,0.005,...,180,1714011,0.800,0.533,0.29974315313218426005,State Procured,Lagging,Lagging,False,True
59,HI,2019,0E-20,1.00000000000000000000,0.000,0.000,0.000,None,171502.00000000000000000000,NaN,...,1,171502,0.000,0.000,0.25334981516250539350,State Procured,Lagging,Lagging,False,True
79,IN,2019,51.92968750000000000000,237.07031250000000000000,0.180,0.045,0.092,216381.91573012832854863723,781749.08426987167145136277,0.217,...,289,998131,0.737,0.682,0.53655690677210387740,District-procured,Lagging,Lagging,False,True
84,KS,2019,144.01418439716312056624,141.98581560283687943376,0.504,0.143,0.143,165734.53386238353796584358,320863.46613761646203415642,0.341,...,286,486598,0.927,0.839,0.96758609971520107255,District-procured,Majority Meeting,Normal,True,False
89,KY,2019,1.01169590643274853873,171.98830409356725146127,0.006,-0.006,0.012,6913.78344628712387210768,666555.21655371287612789232,0.010,...,173,673469,0.821,0.543,0.27843519420854796050,State Procured,Lagging,Lagging,False,True


In [16]:
##code for reviewing states_df (no actual output)
sorted_states_df = current_states_df.sort_values(by='max_year_pct_increase',ascending=False)


print(sorted_states_df[['state_code','pct_districts_meeting_1mbps','max_year_pct_increase','pct_rural','pct_small']])
##extra cols
#,'pct_students_meeting_1mbps','median_ia_bandwidth_per_student_kbps','median_ia_monthly_cost_per_mbps'

    state_code  pct_districts_meeting_1mbps  max_year_pct_increase  pct_rural  \
14          AR                        1.000                  0.919      0.872   
209         SD                        0.966                  0.671      0.980   
144         ND                        0.635                  0.503      0.971   
254         WY                        0.688                  0.411      0.958   
44          DE                        0.412                  0.333      0.471   
244         WI                        0.539                  0.224      0.776   
169         NV                        0.588                  0.212      0.824   
219         TX                        0.436                  0.205      0.824   
224         UT                        0.375                  0.188      0.634   
184         OK                        0.565                  0.181      0.939   
189         OR                        0.472                  0.169      0.815   
234         VT              

In [17]:
output_df = current_states_df[['state_code','procurement_pattern','meeting_category','growth_category','model_state','lagging_state']]
output_df.to_csv('id4003_states_categories.csv', index=False)